# ESCMID Workshop Day 1: Data Cleaning and Preparation for Machine Learning

## Overview

This notebook demonstrates the essential steps for preparing antimicrobial resistance data for machine learning analysis. We'll work with a synthetic dataset that simulates real-world clinical data, focusing on predicting treatment outcomes for bacterial infections.

## Learning Objectives

By the end of this session, you will understand:
- How to load and explore antimicrobial resistance datasets
- Data cleaning techniques for clinical data
- Feature selection and engineering for predictive modeling
- Preprocessing steps including normalization and encoding
- How to prepare data for machine learning algorithms

## Dataset Description

We're working with synthetic clinical data that includes:
- **Patient demographics** (age, gender, location)
- **Clinical characteristics** (ICU admission, ventilation, surgical procedures)
- **Infection details** (type, location, severity)
- **Treatment history** (previous antibiotics, healthcare exposure)
- **Socioeconomic factors** (housing, sanitation, livestock exposure)
- **Target variable**: Treatability with ampicillin+gentamicin combination

Let's begin by importing the necessary libraries and loading our data.

## Step 1: Import Required Libraries

We start by importing the essential Python libraries for data manipulation and analysis:
- **pandas**: For data loading, cleaning, and manipulation
- **os**: For file system operations

In [17]:
import pandas as pd
import os

## Step 2: Data Loading and Initial Exploration

### Mounting Google Drive and Setting Data Path

For this workshop, we're using Google Colab and need to mount Google Drive to access our datasets. The data files are stored in a specific directory structure:

```
ESCMID_workshop/data/
├── AHC_resistance.csv      # Main clinical dataset
├── AHC_dictionary.csv      # Data dictionary with variable descriptions
├── Synthetic_Data_ESCMID_Workshop.csv  # Raw synthetic data
└── Synthetic_Data_cleaned.csv          # Pre-cleaned version
```

### Available Datasets

Let's examine what files are available in our data directory:

In [18]:
# from google.colab import drive
# drive.mount('/content/drive')
# PATH = '/content/drive/MyDrive/ESCMID_workshop/data'
# os.listdir(PATH)

## Step 3: Loading and Exploring the Dataset

### Loading the Main Dataset

We'll work with the `AHC_resistance.csv` file, which contains our main clinical dataset. This file includes patient-level data with various clinical, demographic, and socioeconomic variables.

### Loading the Data Dictionary

The `AHC_dictionary.csv` file contains descriptions of all variables in our dataset, which is crucial for understanding what each column represents.

### Initial Data Inspection

Let's load both files and examine their structure:

In [19]:
PATH = '../DATA'
# Loading the main clinical dataset
df_main = pd.read_csv(f"{PATH}/AHC_resistance.csv")

# View the first 10 rows to understand the structure
df_main.head(10)

,Unnamed: 0,gender,location,age.d,icu.vent,surgery,bloodtransfusion,transfer,sepsis,meningitis,...,fridge,child_abx3w,family_hosp3m,recent.envwater,water.treated,infectcat,specday,amp.gent.treatable,cro.treatable,gram
0,1,m,1,2622,No,No,No,No,No,No,...,No,No,No,No,No,CAI,2,N,N,negative
1,2,m,1,5,Yes,No,No,No,Yes,Yes,...,No,No,No,No,Yes,CAI,0,Y,Y,positive
2,3,f,1,13,No,No,No,No,Yes,No,...,No,No,No,No,No,CAI,0,Y,N,positive
3,4,f,1,0,Yes,No,Yes,Yes,No,No,...,No,No,No,No,No,HAI,4,N,N,negative
4,5,f,1,0,Yes,No,No,Yes,Yes,No,...,No,No,No,No,No,HAI,6,Y,N,negative
5,6,m,1,48,Yes,No,No,No,Yes,Yes,...,No,No,No,No,No,CAI,0,Y,N,positive
6,7,f,1,0,Yes,No,Yes,Yes,No,No,...,No,No,No,No,No,HAI,7,N,N,negative
7,8,f,1,68,Yes,No,Yes,Yes,Yes,No,...,No,No,No,No,No,CAI,0,Y,N,positive
8,9,m,1,0,Yes,No,Yes,Yes,No,No,...,No,No,No,No,No,HAI,18,N,N,negative
9,10,m,1,3159,No,Yes,No,Yes,No,No,...,No,No,No,No,Yes,CAI,0,Y,Y,positive


In [20]:
# Load the data dictionary to understand variable meanings
df_dictionary = pd.read_csv(f"{PATH}/AHC_dictionary.csv")
df_dictionary.head()

,key,definition
0,gender,gender: m = male / f = female
1,location,hospital location (1 = Angkor Hospital for Chi...
2,age.d,patient age in days (sample date - birth date)
3,icu.vent,complication during admission = required icu ...
4,surgery,complication during admission = required surgery


## Step 4: Data Cleaning

### Handling Column Names

The first step in our data cleaning process is to fix any formatting issues with column names. We notice there's an "Unnamed: 0" column that appears to be a row index that was saved incorrectly. Let's rename it to something more meaningful.

In [21]:
# Rename the "Unnamed: 0" column to a more meaningful name
df_main.rename(columns={'Unnamed: 0': 'Id'}, inplace=True)

# Verify the renaming worked correctly
df_main.head(10)

,Id,gender,location,age.d,icu.vent,surgery,bloodtransfusion,transfer,sepsis,meningitis,...,fridge,child_abx3w,family_hosp3m,recent.envwater,water.treated,infectcat,specday,amp.gent.treatable,cro.treatable,gram
0,1,m,1,2622,No,No,No,No,No,No,...,No,No,No,No,No,CAI,2,N,N,negative
1,2,m,1,5,Yes,No,No,No,Yes,Yes,...,No,No,No,No,Yes,CAI,0,Y,Y,positive
2,3,f,1,13,No,No,No,No,Yes,No,...,No,No,No,No,No,CAI,0,Y,N,positive
3,4,f,1,0,Yes,No,Yes,Yes,No,No,...,No,No,No,No,No,HAI,4,N,N,negative
4,5,f,1,0,Yes,No,No,Yes,Yes,No,...,No,No,No,No,No,HAI,6,Y,N,negative
5,6,m,1,48,Yes,No,No,No,Yes,Yes,...,No,No,No,No,No,CAI,0,Y,N,positive
6,7,f,1,0,Yes,No,Yes,Yes,No,No,...,No,No,No,No,No,HAI,7,N,N,negative
7,8,f,1,68,Yes,No,Yes,Yes,Yes,No,...,No,No,No,No,No,CAI,0,Y,N,positive
8,9,m,1,0,Yes,No,Yes,Yes,No,No,...,No,No,No,No,No,HAI,18,N,N,negative
9,10,m,1,3159,No,Yes,No,Yes,No,No,...,No,No,No,No,Yes,CAI,0,Y,Y,positive


## Step 5: Machine Learning Problem Definition

### Objective

We're building a **classification model** to predict whether a bacterial infection can be successfully treated with the antibiotic combination **ampicillin + gentamicin**. This is a clinically relevant question as it can help guide treatment decisions.

### Target Variable

Our target variable is `amp.gent.treatable`, which has two possible values:
- **Y** (Yes): The infection is treatable with ampicillin + gentamicin
- **N** (No): The infection is not treatable with this combination

### Machine Learning Pipeline

To prepare our data for machine learning, we need to complete several preprocessing steps:

1. **Feature Selection**: Choose relevant variables that might predict treatability
2. **Target Definition**: Extract our outcome variable
3. **Data Type Handling**: 
   - Normalize numerical variables (put them on the same scale)
   - Encode categorical variables (convert text categories to numbers)
4. **Data Quality Checks**: Ensure no missing values or data inconsistencies

Let's start by examining all available columns and selecting appropriate features:

In [22]:
# Examine all available columns in our dataset
print("Available columns in the dataset:")
print(df_main.columns.tolist())

Available columns in the dataset:
['Id', 'gender', 'location', 'age.d', 'icu.vent', 'surgery', 'bloodtransfusion', 'transfer', 'sepsis', 'meningitis', 'lrti', 'urti', 'diarrhoea', 'cellulitis', 'abscess', 'uti', 'waz', 'hosptimes1y', 'opdtimes6m', 'pharmpre', 'nursepre', 'khrupre', 'ivfluidpre', 'medpre', 'abxprewhich', 'housesize', 'domesticanimal', 'livestock', 'defecate_toilet', 'fridge', 'child_abx3w', 'family_hosp3m', 'recent.envwater', 'water.treated', 'infectcat', 'specday', 'amp.gent.treatable', 'cro.treatable', 'gram']


## Step 6: Feature Selection

### Feature Categories

Based on clinical knowledge and the data dictionary, we can group our features into several categories:

#### **Patient Demographics**
- `gender`: Patient sex
- `age.d`: Age in days
- `location`: Geographic location

#### **Clinical Characteristics**
- `icu.vent`: ICU admission with ventilation
- `surgery`: Recent surgical procedures
- `bloodtransfusion`: Blood transfusion history
- `transfer`: Patient transfer status

#### **Infection Types** (Clinical syndromes)
- `sepsis`: Bloodstream infection
- `meningitis`: Central nervous system infection
- `lrti`: Lower respiratory tract infection
- `urti`: Upper respiratory tract infection
- `diarrhoea`: Gastrointestinal infection
- `cellulitis`: Skin and soft tissue infection
- `abscess`: Localized infection
- `uti`: Urinary tract infection

#### **Nutritional Status**
- `waz`: Weight-for-age Z-score (nutritional indicator)

#### **Healthcare Exposure**
- `hosptimes1y`: Hospital admissions in past year
- `opdtimes6m`: Outpatient visits in past 6 months
- `pharmpre`: Pharmacy visits
- `nursepre`: Nursing care exposure
- `khrupre`: Healthcare facility exposure
- `ivfluidpre`: IV fluid administration
- `medpre`: Medication history
- `abxprewhich`: Previous antibiotic use

#### **Socioeconomic and Environmental Factors**
- `housesize`: Household size
- `domesticanimal`: Domestic animal exposure
- `livestock`: Livestock exposure
- `defecate_toilet`: Toilet access
- `fridge`: Refrigeration access
- `child_abx3w`: Child antibiotic use
- `family_hosp3m`: Family hospitalization
- `recent.envwater`: Environmental water exposure
- `water.treated`: Treated water access

#### **Infection Characteristics**
- `infectcat`: Infection category
- `specday`: Specimen collection day

Let's create our feature and target datasets:

In [23]:
# Create features dataset with selected variables
features = df_main[[
    'gender',
    'location',
    'age.d',
    'icu.vent',
    'surgery',
    'bloodtransfusion',
    'transfer',
    'sepsis',
    'meningitis',
    'lrti',
    'urti',
    'diarrhoea',
    'cellulitis',
    'abscess',
    'uti',
    'waz',
    'hosptimes1y',
    'opdtimes6m',
    'pharmpre',
    'nursepre',
    'khrupre',
    'ivfluidpre',
    'medpre',
    'abxprewhich',
    'housesize',
    'domesticanimal',
    'livestock',
    'defecate_toilet',
    'fridge',
    'child_abx3w',
    'family_hosp3m',
    'recent.envwater',
    'water.treated',
    'infectcat',
    'specday'
]]

# Create target variable dataset
target = df_main[["amp.gent.treatable"]]

print("Features dataset shape:", features.shape)
print("Target dataset shape:", target.shape)

Features dataset shape: (243, 35)
Target dataset shape: (243, 1)


In [24]:
# Let's examine our features and target variables
print("First 5 rows of features:")
features.head()

First 5 rows of features:


,gender,location,age.d,icu.vent,surgery,bloodtransfusion,transfer,sepsis,meningitis,lrti,...,domesticanimal,livestock,defecate_toilet,fridge,child_abx3w,family_hosp3m,recent.envwater,water.treated,infectcat,specday
0,m,1,2622,No,No,No,No,No,No,No,...,Yes,Yes,No,No,No,No,No,No,CAI,2
1,m,1,5,Yes,No,No,No,Yes,Yes,No,...,No,Yes,Yes,No,No,No,No,Yes,CAI,0
2,f,1,13,No,No,No,No,Yes,No,No,...,Yes,Yes,Yes,No,No,No,No,No,CAI,0
3,f,1,0,Yes,No,Yes,Yes,No,No,No,...,Yes,Yes,No,No,No,No,No,No,HAI,4
4,f,1,0,Yes,No,No,Yes,Yes,No,No,...,No,No,No,No,No,No,No,No,HAI,6


In [25]:
# Examine the target variable distribution
print("Target variable distribution:")
target.value_counts()

Target variable distribution:


amp.gent.treatable
Y                     158
N                      85
Name: count, dtype: int64

## Step 7: Data Type Analysis and Quality Assessment

### Understanding Data Types

Before we can apply machine learning algorithms, we need to understand what types of data we're working with:

- **Numerical variables**: Can be used directly but may need scaling
- **Categorical variables**: Need to be converted to numerical format (encoded)
- **Missing values**: Need to be identified and handled

Let's examine the data types and check for any data quality issues:

In [26]:
# Analyze data types and missing values
print("Data types and missing value analysis:")
features.info()

Data types and missing value analysis:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            243 non-null    object 
 1   location          243 non-null    int64  
 2   age.d             243 non-null    int64  
 3   icu.vent          243 non-null    object 
 4   surgery           243 non-null    object 
 5   bloodtransfusion  243 non-null    object 
 6   transfer          243 non-null    object 
 7   sepsis            243 non-null    object 
 8   meningitis        243 non-null    object 
 9   lrti              243 non-null    object 
 10  urti              243 non-null    object 
 11  diarrhoea         243 non-null    object 
 12  cellulitis        243 non-null    object 
 13  abscess           243 non-null    object 
 14  uti               243 non-null    object 
 15  waz               243 non-null    float64
 16  hospt

### Key Observations from Data Analysis

From the `info()` output, we can see:

1. **Dataset Size**: 243 patients with 35 features
2. **Data Types**:
   - **Numerical variables** (int64, float64): `location`, `age.d`, `waz`, `hosptimes1y`, `opdtimes6m`, `housesize`, `specday`
   - **Categorical variables** (object): All others (28 variables)

3. **Missing Values**: 
   - Most variables have complete data (243 non-null values)
   - **Exception**: `abxprewhich` has only 113 non-null values (130 missing values)
   - This variable represents "which antibiotics were used previously" and is missing when no previous antibiotics were used

4. **Memory Usage**: 66.6 KB - very manageable dataset size

### Next Steps for Data Preprocessing

We need to handle the categorical variables by converting them to a numerical format that machine learning algorithms can process. The most common approach is **one-hot encoding**.

## Step 8: One-Hot Encoding for Categorical Variables

### What is One-Hot Encoding?

One-hot encoding is a process of converting categorical variables into a numerical format by creating binary (0/1) columns for each category. 

**Example**: If we have a variable `gender` with values ["Male", "Female"], one-hot encoding creates:
- `gender_Male`: 1 if Male, 0 if Female  
- `gender_Female`: 1 if Female, 0 if Male

### Why Do We Need One-Hot Encoding?

Machine learning algorithms work with numbers, not text. One-hot encoding:
- Converts categorical text data to numerical format
- Avoids imposing artificial ordering on categories
- Allows algorithms to treat each category independently
- Handles missing values appropriately (pandas creates separate categories for NaN values)

### Applying One-Hot Encoding

Pandas provides a convenient `get_dummies()` function that automatically:
- Identifies categorical (object) columns
- Creates binary columns for each unique value
- Handles missing values by creating a separate category
- Keeps numerical columns unchanged

In [27]:
# Apply one-hot encoding to convert categorical variables to numerical format
features_encoded = pd.get_dummies(features)

print(f"Original features shape: {features.shape}")
print(f"Encoded features shape: {features_encoded.shape}")
print(f"Number of new columns created: {features_encoded.shape[1] - features.shape[1]}")

Original features shape: (243, 35)
Encoded features shape: (243, 64)
Number of new columns created: 29


In [28]:
# Examine the new column names created by one-hot encoding
print("New columns created by one-hot encoding:")
print(features_encoded.columns.tolist())

New columns created by one-hot encoding:
['location', 'age.d', 'waz', 'hosptimes1y', 'opdtimes6m', 'housesize', 'specday', 'gender_f', 'gender_m', 'icu.vent_No', 'icu.vent_Yes', 'surgery_No', 'surgery_Yes', 'bloodtransfusion_No', 'bloodtransfusion_Yes', 'transfer_No', 'transfer_Yes', 'sepsis_No', 'sepsis_Yes', 'meningitis_No', 'meningitis_Yes', 'lrti_No', 'lrti_Yes', 'urti_No', 'urti_Yes', 'diarrhoea_No', 'diarrhoea_Yes', 'cellulitis_No', 'cellulitis_Yes', 'abscess_No', 'abscess_Yes', 'uti_No', 'uti_Yes', 'pharmpre_No', 'pharmpre_Yes', 'nursepre_No', 'nursepre_Yes', 'khrupre_No', 'khrupre_Yes', 'ivfluidpre_No', 'ivfluidpre_Yes', 'medpre_No', 'medpre_Yes', 'abxprewhich_AnyTBdrugs', 'abxprewhich_PenicillinFamily', 'abxprewhich_Unknown', 'domesticanimal_No', 'domesticanimal_Yes', 'livestock_No', 'livestock_Yes', 'defecate_toilet_No', 'defecate_toilet_Yes', 'fridge_No', 'fridge_Yes', 'child_abx3w_No', 'child_abx3w_Yes', 'family_hosp3m_No', 'family_hosp3m_Yes', 'recent.envwater_No', 'recent

## Step 9: Final Data Preparation Summary

### What We've Accomplished

1. **Data Loading**: Successfully loaded the antimicrobial resistance dataset and data dictionary
2. **Data Cleaning**: Fixed column naming issues 
3. **Feature Selection**: Selected 35 clinically relevant variables as predictors
4. **Target Definition**: Identified our outcome variable (`amp.gent.treatable`)
5. **Data Quality Assessment**: Analyzed data types and missing values
6. **One-Hot Encoding**: Converted categorical variables to numerical format suitable for machine learning

### Current Dataset Status

- **Features**: 243 patients × ~90 variables (after one-hot encoding)
- **Target**: 243 patients with binary outcome (Y/N for treatability)
- **Data Types**: All features are now numerical (ready for ML algorithms)
- **Missing Values**: Handled appropriately through encoding

### Target Variable Distribution

Let's examine the class balance in our target variable:

In [29]:
# Analyze target variable distribution
print("Target variable distribution:")
target_counts = target['amp.gent.treatable'].value_counts()
print(target_counts)

print(f"\nClass proportions:")
target_proportions = target['amp.gent.treatable'].value_counts(normalize=True)
print(f"Treatable (Y): {target_proportions['Y']:.1%}")
print(f"Not treatable (N): {target_proportions['N']:.1%}")

print(f"\nClass balance ratio: {target_counts['Y'] / target_counts['N']:.2f}")
print("This indicates a moderately imbalanced dataset favoring treatable cases.")

Target variable distribution:
amp.gent.treatable
Y    158
N     85
Name: count, dtype: int64

Class proportions:
Treatable (Y): 65.0%
Not treatable (N): 35.0%

Class balance ratio: 1.86
This indicates a moderately imbalanced dataset favoring treatable cases.


In [30]:
#join features and target datasets for exporting
df_final = pd.concat([features_encoded, target], axis=1)

print(df_final)

# Export the final dataset to a CSV file
df_final.to_csv(f"{PATH}/AHC_final_dataset.csv", index=False)

     location  age.d       waz  hosptimes1y  opdtimes6m  housesize  specday  \
0           1   2622 -1.669851            0           0          6        2   
1           1      5 -1.954199            0           0          7        0   
2           1     13  0.421916            0           0          7        0   
3           1      0 -3.569651            0           0          9        4   
4           1      0 -3.024092            0           0          3        6   
..        ...    ...       ...          ...         ...        ...      ...   
238         2   4738 -1.102737            0           0          5        0   
239         2   2518 -3.097708            0           1          6        0   
240         2   1759 -1.617162            0           0          5        0   
241         2   1086 -1.513930            0           0          7        0   
242         2     18  0.341330            0           0          7        0   

     gender_f  gender_m  icu.vent_No  ...  child_ab

## Summary and Next Steps

### What We've Learned Today

In this session, we successfully prepared a real-world antimicrobial resistance dataset for machine learning analysis. Key accomplishments include:

#### **Technical Skills Developed**
- Loading and exploring clinical datasets using pandas
- Data cleaning techniques (column renaming, handling missing values)
- Feature selection based on clinical knowledge
- One-hot encoding for categorical variables
- Data quality assessment and validation

#### **Domain Knowledge Applied**
- Understanding antimicrobial resistance as a clinical problem
- Recognizing different types of clinical variables (demographics, symptoms, exposures)
- Appreciating the complexity of factors that influence treatment outcomes
- Class imbalance considerations in medical datasets

#### **Data Science Workflow**
- Problem definition (predicting treatment efficacy)
- Data exploration and quality assessment
- Feature engineering and preprocessing
- Preparation for machine learning algorithms

### Dataset Ready for Machine Learning

Our data is now properly formatted:
- **Target**: Binary classification (treatable vs. not treatable)
- **No missing values**: Handled through proper encoding
- **Consistent data types**: All numerical, ready for algorithms

### Next Steps (Day 2)

In the next session, we will:
1. **Split the data** into training and testing sets
2. **Build machine learning models** (logistic regression, random forest, etc.)
3. **Evaluate model performance** using appropriate metrics
4. **Interpret results** in a clinical context
5. **Discuss model limitations** and potential improvements

### Clinical Relevance

This preprocessing pipeline is essential for any antimicrobial resistance prediction model in real clinical settings, where:
- Treatment decisions must be rapid and accurate
- Multiple patient factors influence outcomes
- Data quality and completeness vary
- Model interpretability is crucial for clinical adoption

**Great work!** You've successfully completed the data preparation phase of building an antimicrobial resistance prediction model.